In [8]:
#!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 45.9 MB/s eta 0:00:00m eta 0:00:010:01:01


In [1]:
import os
import cv2
import copy
import time
import torch
#import library
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from os.path import join
import torch.optim as optim
from torchmetrics import AUROC
import torch.utils.data as data
from torchmetrics import F1Score
from matplotlib import pyplot as plt
from torchmetrics.functional import auroc
import torchvision.transforms as transforms
from torchmetrics.functional import precision_recall

## Data

In [2]:
RESULTS_DIR = os.path.join('/home',
                            'emily',
                            'Desktop',
                            'CAD',
                            'results')


DATA_DIR = os.path.join('/home',
                        'emily',
                        'Desktop',
                        'CAD',
                        'challenge1')

In [3]:
torch.cuda.is_available()

True

In [4]:
# hyperparameters
batch_size = 2
learning_rate = 0.001
epochs = 30
momentum = 0.9
lr_step_size = 6   # if < epochs, we are using decaying learning rate
lr_gamma = 0.1
data_augmentation = True
dropout = 0.5
activation = nn.ReLU()

# make visible only one GPU at the time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # <-- should be the ID of the GPU you want to use

# options
# device = "cuda:0"           # put here "cuda:0" if you want to run on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
monitor_display = True      # whether to display monitored performance plots
display_first_n = 0         # how many samples/batches are displayed
num_workers = 2             # how many workers (=threads) for fetching data
pretrained = False          # whether to test a pretrained model (to be loaded) or train a new one
display_errors = True       # whether to display errors (only in pretrained mode)

In [5]:
import torchvision.models as models

# pretrained resnet 50
resnet50 = models.resnet50(pretrained=True)

# freezing the first 10 layers
ct = 0
# model.children() which returns layers.
for child in resnet50.children():
 ct += 1
 if ct < 3:
     for param in child.parameters():
      # True means it will be backpropagrated
      # to freeze a layer you need to set requires_grad to False for all parameters of a layer. 
       param.requires_grad = False

# change last fully convolutional layer so it returns just 2 classes
resnet50.fc = nn.Linear(2048,2)

/home/emily/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/emily/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/emily/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


### GENERATING FEATURES